In [ ]:
%run InitialDataCleanup.ipynb
IPython.display.clear_output()

In [ ]:
df = result
df.tail()

In [ ]:
#target_names = ['contclaims_count_regular']
target_names = list(df.columns)

In [ ]:
split_fraction = 0.7
train_split = int(split_fraction * int(df.shape[0]))
step = 1

past = 20
future = 1
learning_rate = 0.001
batch_size = 12

sequence_length = int(past / step)

In [ ]:
titles = list(df.columns)
features = df[titles]
features.head()

In [ ]:
start = past + future
end = start + train_split

x_data = features.values[0: -start]
y_data = features[target_names].values[start:]

x_train = x_data[:train_split]
x_val = x_data[train_split:]

y_train = y_data[:train_split]
y_val = y_data[train_split:]

x_scaler = MinMaxScaler()
x_train_scaled = x_scaler.fit_transform(x_train)
x_val_scaled = x_scaler.transform(x_val)

y_scaler = MinMaxScaler()
y_train_scaled = y_scaler.fit_transform(y_train)
y_val_scaled = y_scaler.transform(y_val)

len(x_train) + len(x_val), len(y_train) + len(y_val), len(features)

In [ ]:
dataset_train = keras.preprocessing.timeseries_dataset_from_array(
    x_train_scaled,
    y_train_scaled,
    sequence_length=sequence_length,
    sampling_rate=step,
    batch_size=batch_size,
)

In [ ]:
dataset_val = keras.preprocessing.timeseries_dataset_from_array(
    x_val_scaled,
    y_val_scaled,
    sequence_length=sequence_length,
    sampling_rate=step,
    batch_size=batch_size,
)

In [ ]:
for batch in dataset_train.take(1):
    inputs, targets = batch
print("Input shape:", inputs.numpy().shape)
print("Target shape:", targets.numpy().shape)

print('dataset val')
i = 0
for j in dataset_val:
    x, y = j
    print('x shape:', x.shape)
    print('y shape:', y.shape)
    i = i + 1

print(f'length of x_val: {len(x_val)}')
print(f'length of validation data: {len(dataset_val)}')
print(f'number of batches: {i}')

In [ ]:
model = tf.keras.Sequential([
    keras.layers.Input(shape=(inputs.shape[1], inputs.shape[2])),
    keras.layers.LSTM(32, activation='tanh'),
    keras.layers.Dense(len(target_names))
])
model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate), loss="mse")
model.summary()

In [ ]:

%load_ext tensorboard
import os
import shutil
from datetime import datetime

path_checkpoint = "covid_model_checkpoint.h5"
es_callback = keras.callbacks.EarlyStopping(monitor="val_loss", min_delta=0, patience=5)

epochs = 50

modelckpt_callback = keras.callbacks.ModelCheckpoint(
    monitor="val_loss",
    filepath=path_checkpoint,
    verbose=1,
    save_weights_only=True,
    save_best_only=True,
)

if os.path.exists('./logs'):
    shutil.rmtree('./logs/')
log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

history = model.fit(
    dataset_train,
    epochs=epochs,
    validation_data=dataset_val,
    callbacks=[es_callback, modelckpt_callback, tensorboard_callback],
)